In [9]:
from sim.simulator import *
from sim.sim_controllers import *
from sim.config import ExperimentConfig
from sim.motor_controllers import *
from utils.path_utils import join_paths
from utils.gui_utils import UserPrompt

# Configure the Simulation Parameters

In [4]:
################################ User Input ################################
base_path = f"data/Exp1_GuyGilad_logs_yolo/" # path of the logs (containing exoeriment_config and bbox log files)
exp_base_path = "logs" # path to save the experiment logs
############################################################################

log_path = join_paths(base_path, "bboxes2.csv")

In [5]:

# Load the experiment configuration
experiment_config = ExperimentConfig.load_json(f"{base_path}/experiment_config.json")

In [6]:
################################ User Input ################################
# initialize the logging configuration
log_config = LogConfig(
    root_folder=exp_base_path,
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
)

# initialize the timing configuration
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.32, 0.32),
    experiment_config=experiment_config,
)
############################################################################

# Choose a Controller
Run the cell of the controller you need

**MLP Controller**

In [13]:
################################ User Input ################################
model_path = None # path to the model file, if None, the user will be prompted to select the file
############################################################################

import torch


model_path = UserPrompt.open_file(path=model_path)

model:WormPredictor = torch.load(model_path)
controller = MLPController(time_config, log_path, model, model.io_config)


**Optimal Controller**

In [ ]:
controller = OptimalController(time_config, log_path)

**CSV Controller**

In [ ]:
controller = CsvController(time_config, log_path)

**If you want to log the Simulation run the cell below as well**

In [ ]:
controller = LoggingController(controller, log_config)

# Run The Simulation

In [ ]:
motor_controller = SineMotorController(time_config)
sim = Simulator(time_config, experiment_config, controller, reader=None, motor_controller=motor_controller)

In [ ]:
sim.run()

time_config.save_json(exp_base_path + "/time_config.json")
experiment_config.save_json(exp_base_path + "/exp_config.json")

# Show the Results

In [ ]:
from eval.analysis import Plotter

pltr = Plotter(log_config.bbox_file_path, time_config)
data = pltr.print_statistics(n=10)

In [ ]:
pltr.plot_area_vs_speed()

In [ ]:
_ = pltr.plot_deviation()